In [72]:
import pulp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
distance_matrix = pd.read_csv('dataset/Distance_Matrix.csv').drop('Unnamed: 0', axis=1)
biomass_history = pd.read_csv('dataset/Biomass_History.csv')


### Detalhes:

- *Variáveis de Decisão*: As variáveis `x`, `y`, `depot`, e `refinery` representam as decisões de alocação e seleção.
- *Função Objetivo*: A função objetivo combina o custo de transporte e a incompatibilidade da previsão de biomassa.
- *Restrições*: As restrições garantem que cada local de colheita seja atribuído a um depósito, cada depósito a uma refinaria, e outras restrições específicas do problema.

Este código é um esboço geral e pode precisar de ajustes e detalhamento adicionais com base nos dados e requisitos específicos do problema.

In [136]:
year = '2011'
B = biomass_history[year].to_numpy()
D = distance_matrix.to_numpy()
C = 20_000

num_warehouses = 25
num_harvesting_sites = 2418

In [137]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417
0,0.0000,11.3769,20.4557,38.1227,45.3810,54.9915,78.6108,118.6750,102.6639,113.4309,...,683.8771,687.6310,697.3246,669.3962,667.6788,665.5775,662.0291,665.9655,673.2073,681.4235
1,11.3769,0.0000,9.0788,28.9141,36.1724,45.7829,69.4022,78.2329,93.4553,111.1832,...,681.6295,685.3833,695.0769,667.1485,665.4311,663.3298,659.7815,663.7178,670.9596,679.1758
2,20.4557,9.0788,0.0000,22.3791,29.6374,39.2478,62.8671,71.6979,86.9203,111.7859,...,682.2323,685.9861,695.6796,667.7513,666.0339,663.9326,660.3843,664.3206,671.5623,679.7786
3,38.1227,28.9141,22.3791,0.0000,11.8343,23.5413,41.8396,50.6703,65.8927,82.5852,...,681.4226,685.1765,694.8701,666.9417,665.2243,663.1230,659.5746,663.5110,670.7528,678.9690
4,45.3810,36.1724,29.6374,11.8343,0.0000,11.7070,24.3986,33.2293,53.9901,65.1442,...,663.9816,667.7355,677.4291,649.5007,647.7833,645.6820,642.1336,646.0700,653.3118,661.5280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,671.2005,668.9528,669.5556,669.9084,652.1102,640.4032,627.7116,622.9663,638.9532,617.7221,...,109.0007,112.7545,122.4481,12.3091,9.1558,0.0000,14.5629,22.3860,36.6284,44.8446
2414,663.4455,661.1978,661.8006,662.1534,644.3551,632.6481,619.9566,615.2112,631.1981,609.9671,...,101.2456,104.9995,114.6931,26.4955,23.3422,14.1864,0.0000,14.5984,28.8407,37.0570
2415,663.7748,661.5271,662.1298,662.4826,644.6844,632.9774,620.2859,615.5405,631.5274,610.2964,...,84.1469,87.9008,97.5943,34.3098,31.1565,22.0007,12.8105,0.0000,14.2423,22.4586
2416,671.0165,668.7688,669.3715,669.7244,651.9261,640.2191,627.5276,622.7822,638.7691,617.5381,...,91.3886,95.1425,104.8361,48.5521,45.3988,36.2430,27.0528,14.2423,0.0000,12.4741


In [138]:
def print_matrix(matrix):
    print(pd.DataFrame(matrix).to_string(index=True, header=True))

print_matrix(D[:5, :5])

         0        1        2        3        4
0   0.0000  11.3769  20.4557  38.1227  45.3810
1  11.3769   0.0000   9.0788  28.9141  36.1724
2  20.4557   9.0788   0.0000  22.3791  29.6374
3  38.1227  28.9141  22.3791   0.0000  11.8343
4  45.3810  36.1724  29.6374  11.8343   0.0000


In [139]:
import pulp

warehouse_locations = [i for i in range(num_possible_warehouses_locations)]  # Creates a list of all warehouses
# Creates a list of all demand nodes
harvest_sites = [i for i in range(num_harvesting_sites)]

cost_matrix = D * B

In [140]:
# The cost data is made into a dictionary
costs = pulp.makeDict([harvest_sites, warehouse_locations], cost_matrix, 0)

In [141]:
warehouses = pulp.LpVariable.dicts("W%s", warehouse_locations, cat=pulp.LpBinary)
serv_harvest=pulp.LpVariable.dicts("H%s_W%s", (harvest_sites, warehouse_locations), cat=pulp.LpBinary)

In [142]:
# Creates the 'prob' variable to contain the problem data
prob = pulp.LpProblem("Supply_Chain_Optimization", pulp.LpMinimize)

cost_of_transportation = pulp.lpSum([serv_harvest[h][w] * costs[h][w] for w in warehouse_locations for h in harvest_sites])
cost_of_underutilization = pulp.lpSum([C - pulp.lpSum([serv_harvest[h][w] for w in warehouse_locations]) for h in harvest_sites])
a= 0.01
b=1
prob += a * cost_of_transportation + b * cost_of_underutilization

In [143]:
# Constraints

# Constraint 1: Number of warehouses must be less or equal to 5
prob += (
    pulp.lpSum([warehouses[w] for w in warehouse_locations]) <= num_warehouses, 
    "Maximum_number_of_warehouses"
    )

# Constraint 2: Each harvesting site must be assigned to exactly one warehouse
for h in harvest_sites:
    prob += (
        pulp.lpSum([serv_harvest[h][w] for w in warehouse_locations]) == 1,
        f"Harvesting_site_{h}_must_be_assigned_to_exactly_one_warehouse"
    )

# Constraint 3: If a harvesting site is assigned to a warehouse, the warehouse must be selected
for h in harvest_sites:
    for w in warehouse_locations:
        prob += (
            serv_harvest[h][w] <= warehouses[w],
            f"Harvesting_site_{h}_must_be_assigned_to_warehouse_{w}_only_if_warehouse_{w}_is_selected"
        )


In [144]:
prob.solve()
# print the status of the solution
print("Status:", pulp.LpStatus[prob.status])

# print the value of the objective
print("Total Cost of Transportation = ", pulp.value(prob.objective))

# print the value of the variables
for v in prob.variables():
    print(v.name, "=", v.varValue)


Status: Optimal
Total Cost of Transportation =  48358910.04739014
H0_W0 = 1.0
H0_W1 = 0.0
H0_W10 = 0.0
H0_W11 = 0.0
H0_W12 = 0.0
H0_W13 = 0.0
H0_W14 = 0.0
H0_W15 = 0.0
H0_W16 = 0.0
H0_W17 = 0.0
H0_W18 = 0.0
H0_W19 = 0.0
H0_W2 = 0.0
H0_W20 = 0.0
H0_W21 = 0.0
H0_W22 = 0.0
H0_W23 = 0.0
H0_W24 = 0.0
H0_W3 = 0.0
H0_W4 = 0.0
H0_W5 = 0.0
H0_W6 = 0.0
H0_W7 = 0.0
H0_W8 = 0.0
H0_W9 = 0.0
H1000_W0 = 0.0
H1000_W1 = 0.0
H1000_W10 = 0.0
H1000_W11 = 0.0
H1000_W12 = 0.0
H1000_W13 = 1.0
H1000_W14 = 0.0
H1000_W15 = 0.0
H1000_W16 = 0.0
H1000_W17 = 0.0
H1000_W18 = 0.0
H1000_W19 = 0.0
H1000_W2 = 0.0
H1000_W20 = 0.0
H1000_W21 = 0.0
H1000_W22 = 0.0
H1000_W23 = 0.0
H1000_W24 = 0.0
H1000_W3 = 0.0
H1000_W4 = 0.0
H1000_W5 = 0.0
H1000_W6 = 0.0
H1000_W7 = 0.0
H1000_W8 = 0.0
H1000_W9 = 0.0
H1001_W0 = 0.0
H1001_W1 = 0.0
H1001_W10 = 0.0
H1001_W11 = 0.0
H1001_W12 = 0.0
H1001_W13 = 1.0
H1001_W14 = 0.0
H1001_W15 = 0.0
H1001_W16 = 0.0
H1001_W17 = 0.0
H1001_W18 = 0.0
H1001_W19 = 0.0
H1001_W2 = 0.0
H1001_W20 = 0.0
H1001_W2

In [132]:

# create a dataframe with the solution
solution = pd.DataFrame(columns=['Harvesting Site', 'Warehouse', 'Selected'])
for v in prob.variables():
    if v.varValue == 1 and v.name.startswith('H'):
        splitted = v.name.split('_')
        solution = pd.concat(
            [solution, pd.DataFrame([[splitted[0][1:], splitted[1][1:], v.varValue]], columns=['Harvesting Site', 'Warehouse', 'Selected'])]
            )
#solution = solution[solution['Selected'] == 1].reset_index(drop=True)
solution


,Harvesting Site,Warehouse,Selected
0,0,0,1.0
0,1000,13,1.0
0,1001,13,1.0
0,1002,13,1.0
0,1003,13,1.0
...,...,...,...
0,997,13,1.0
0,998,13,1.0
0,999,13,1.0
0,99,13,1.0


In [145]:
solution['HS_Longitude']= solution['Harvesting Site'].apply(lambda idx: biomass_history['Longitude'][int(idx)])
solution['HS_Latitude']= solution['Harvesting Site'].apply(lambda idx: biomass_history['Latitude'][int(idx)])
solution['W_Longitude']= solution['Warehouse'].apply(lambda idx: biomass_history['Longitude'][int(idx)])
solution['W_Latitude']= solution['Warehouse'].apply(lambda idx: biomass_history['Latitude'][int(idx)])
solution

,Harvesting Site,Warehouse,Selected,HS_Longitude,HS_Latitude,W_Longitude,W_Latitude
0,0,0,1.0,71.33144,24.66818,71.33144,24.66818
0,1000,13,1.0,72.04807,22.81437,72.36657,24.66818
0,1001,13,1.0,72.12769,22.81437,72.36657,24.66818
0,1002,13,1.0,72.20732,22.81437,72.36657,24.66818
0,1003,13,1.0,72.28694,22.81437,72.36657,24.66818
...,...,...,...,...,...,...,...
0,997,13,1.0,71.80919,22.81437,72.36657,24.66818
0,998,13,1.0,71.88882,22.81437,72.36657,24.66818
0,999,13,1.0,71.96844,22.81437,72.36657,24.66818
0,99,13,1.0,71.88882,24.26518,72.36657,24.66818


In [ ]:
unique_warehouses = solution[['Warehouse', 'W_Longitude', 'W_Latitude']].drop_duplicates().reset_index(drop=True)

plt.figure(figsize=(25, 20))
sns.scatterplot(x='HS_Longitude', y='HS_Latitude', data=solution, color='blue')
sns.scatterplot(x='W_Longitude', y='W_Latitude', data=unique_warehouses, color='red')

# draw lines between harvesting sites and warehouses
for i in range(len(solution)):
    plt.plot([solution['HS_Longitude'][i], solution['W_Longitude'][i]], [solution['HS_Latitude'][i], solution['W_Latitude'][i]], color='black', alpha=0.5)

plt.show()



In [225]:
solution['Harvesting Site'].apply(lambda idx: biomass_history['Longitude'][int(idx)])

0     71.33144
1     72.12769
2     72.20732
3     72.28694
4     72.36657
5     71.25181
6     71.33144
7     71.41106
8     71.49069
9     71.57031
10    71.64994
11    71.41106
12    71.72956
13    71.80919
14    71.88882
15    71.96844
16    72.04807
17    71.49069
18    71.57031
19    71.64994
20    71.72956
21    71.80919
22    71.88882
23    71.96844
24    72.04807
Name: Harvesting Site, dtype: float64

In [208]:
biomass_history

,Index,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017
0,0,24.66818,71.33144,8.475744,8.868568,9.202181,6.023070,10.788374,6.647325,7.387925,5.180296
1,1,24.66818,71.41106,24.029778,28.551348,25.866415,21.634459,34.419411,27.361908,40.431847,42.126945
2,2,24.66818,71.49069,44.831635,66.111168,56.982258,53.003735,70.917908,42.517117,59.181629,73.203232
3,3,24.66818,71.57031,59.974419,80.821304,78.956543,63.160561,93.513924,70.203171,74.536720,101.067352
4,4,24.66818,71.64994,14.653370,19.327524,21.928144,17.899586,19.534035,19.165791,16.531315,26.086885
...,...,...,...,...,...,...,...,...,...,...,...
2413,2413,20.15456,72.84432,5.199882,4.516778,4.321080,2.658953,5.113997,5.301668,6.419223,5.321604
2414,2414,20.15456,72.92394,0.122287,0.126717,0.101494,0.111509,0.121749,0.122467,0.145785,0.120626
2415,2415,20.15456,73.00357,0.039415,0.040843,0.032713,0.035941,0.039241,0.039473,0.046989,0.038879
2416,2416,20.15456,73.08319,2.719220,1.370163,0.818687,1.205721,1.316443,1.324201,1.576338,1.304297


In [206]:
solution

,Harvesting Site,Warehouse,Selected
0,0,0,0.0
0,0,1,0.0
0,0,10,1.0
0,0,11,0.0
0,0,12,0.0
...,...,...,...
0,9,5,0.0
0,9,6,0.0
0,9,7,0.0
0,9,8,0.0


In [186]:
# positions